In [5]:
from logging_config import setup_logging
from bhavcopy_utils import file_exists
import os

NSE_base_url = os.getenv('NSE_base_url')
bhav_copy_archive_path = os.getenv("bhav_copy_archive_path")
INDEX_LIST = os.getenv('INDEX_LIST').split(',')
environment = os.getenv('APP_ENV', 'DEBUG')

NSE_PRICE_DATA_OPEN = os.getenv("NSE_PRICE_DATA_OPEN")
NSE_PRICE_DATA_HIGH = os.getenv("NSE_PRICE_DATA_HIGH")
NSE_PRICE_DATA_LOW = os.getenv("NSE_PRICE_DATA_LOW")
NSE_PRICE_DATA_CLOSE = os.getenv("NSE_PRICE_DATA_CLOSE")
NSE_PRICE_DATA_DELIV_QTY = os.getenv("NSE_PRICE_DATA_DELIV_QTY")
NSE_PRICE_DATA_DELIV_PER = os.getenv("NSE_PRICE_DATA_DELIV_PER")
NSE_VOLUME_DATA = os.getenv("NSE_VOLUME_DATA")


logger = setup_logging(logger_name='price_loger',
                       info_file='specific_info.log', 
                        warning_file='specific_warning.log', 
                        error_file='specific_error.log', 
                        environment=environment)

o_logger = h_logger = l_logger = c_logger = v_logger = d_logger = dp_logger = logger

In [8]:
print(NSE_PRICE_DATA_OPEN)

NSE_PRICE_DATA_OPEN.csv


In [13]:
# ========= Column Configurations ========= #
# Base input column names from NSE CSV
base_columns = {
    "open": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' OPEN_PRICE'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'OPEN_PRICE'],
        "pivot_value": 'OPEN_PRICE',
        "outfile": NSE_PRICE_DATA_OPEN,
        "logger": o_logger
    },
    "high": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' HIGH_PRICE'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'HIGH_PRICE'],
        "pivot_value": 'HIGH_PRICE',
        "outfile": NSE_PRICE_DATA_HIGH,
        "logger": h_logger
    },
    "low": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' LOW_PRICE'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'LOW_PRICE'],
        "pivot_value": 'LOW_PRICE',
        "outfile": NSE_PRICE_DATA_LOW,
        "logger": l_logger
    },
    "close": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' CLOSE_PRICE'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'CLOSE_PRICE'],
        "pivot_value": 'CLOSE_PRICE',
        "outfile": NSE_PRICE_DATA_CLOSE,
        "logger": c_logger
    },
    "volume": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' TTL_TRD_QNTY'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'TTL_TRD_QNTY'],
        "pivot_value": 'TTL_TRD_QNTY',
        "outfile": NSE_VOLUME_DATA,
        "logger": v_logger
    },
    "delivery_qty": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' DELIV_QTY'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'DELIV_QTY'],
        "pivot_value": 'DELIV_QTY',
        "outfile": NSE_PRICE_DATA_DELIV_QTY,
        "logger": d_logger
    },
    "delivery_per": {
        "required": ['SYMBOL',' SERIES', ' DATE1', ' DELIV_PER'],
        "renamed": ['SYMBOL', 'SERIES', 'Date', 'DELIV_PER'],
        "pivot_value": 'DELIV_PER',
        "outfile": NSE_PRICE_DATA_DELIV_PER,
        "logger": dp_logger
    }
}


In [14]:
import pandas as pd

def create_or_add_master_data(filepath, date, config_key):
    config = base_columns[config_key]

    print(config)
    df = pd.read_csv(filepath)
    df = df[config["required"]]
    df.columns = config["renamed"]

    # Keep only EQ, BE series
    df = df[(df['SERIES'] == ' EQ') | (df['SERIES'] == ' BE')]
    df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)

    pivot_data = df.pivot(columns='SYMBOL', index='Date', values=config["pivot_value"])
    pivot_data.reset_index(inplace=True)
    pivot_data['Date'] = pd.to_datetime(pivot_data['Date'])

    outfile = config["outfile"]
    print(outfile)
    logger = config["logger"]

    if file_exists(outfile):
        nse_data = pd.read_csv(outfile)
    else:
        nse_data = pd.DataFrame(columns=['Date'] + list(df['SYMBOL'].values))

    if pivot_data.empty:
        logger.critical(f"Pivot data is empty. Please check input. - {date}")
        return False

    # Handle missing new stocks
    missing_columns = set(pivot_data.columns) - set(nse_data.columns)
    if len(missing_columns) > 0:
        for col in missing_columns:
            logger.warning(f"Stock missing in nse_data: {col}  - {date}")
        missing_data = pd.DataFrame({col: pd.NA for col in missing_columns}, index=nse_data.index)
        nse_data = pd.concat([nse_data, missing_data], axis=1)

    # Handle missing old stocks
    missing_columns_pivot = set(nse_data.columns) - set(pivot_data.columns)
    if len(missing_columns_pivot) > 0:
        num_rows = pivot_data.shape[0]
        new_columns_df = pd.DataFrame({col: [np.nan]*num_rows for col in missing_columns_pivot})
        for col in missing_columns_pivot:
            logger.warning(f"Stock missing in pivot_data: {col}  - {date}")
        pivot_data = pd.concat([pivot_data, new_columns_df], axis=1)

    # Merge
    nse_data = pd.concat([nse_data, pivot_data], ignore_index=True)

    nse_data['Date'] = pd.to_datetime(nse_data['Date'])
    nse_data = nse_data.sort_values(by='Date')
    nse_data = nse_data.groupby('Date', as_index=False).first()
    nse_data = nse_data.copy()

    # Fill NaN only for volume-like data
    if config_key in ['volume']:
        nse_data.fillna(0, inplace=True)

    nse_data.to_csv(outfile, index=False)
    return True

In [15]:
filepath = '/Users/shubhgoela/Downloads/sec_bhavdata_full_14082025.csv'

date = '14-08-2025'
create_or_add_master_data(filepath, date, "open")
create_or_add_master_data(filepath, date, "high")
create_or_add_master_data(filepath, date, "low")
create_or_add_master_data(filepath, date, "close")
create_or_add_master_data(filepath, date, "volume")
# create_or_add_master_data(filepath, date, "turnover")
create_or_add_master_data(filepath, date, "delivery_qty")
create_or_add_master_data(filepath, date, "delivery_per")


{'required': ['SYMBOL', ' SERIES', ' DATE1', ' OPEN_PRICE'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'OPEN_PRICE'], 'pivot_value': 'OPEN_PRICE', 'outfile': 'NSE_PRICE_DATA_OPEN.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_OPEN.csv
{'required': ['SYMBOL', ' SERIES', ' DATE1', ' HIGH_PRICE'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'HIGH_PRICE'], 'pivot_value': 'HIGH_PRICE', 'outfile': 'NSE_PRICE_DATA_HIGH.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_HIGH.csv
{'required': ['SYMBOL', ' SERIES', ' DATE1', ' LOW_PRICE'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'LOW_PRICE'], 'pivot_value': 'LOW_PRICE', 'outfile': 'NSE_PRICE_DATA_LOW.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_LOW.csv
{'required': ['SYMBOL', ' SERIES', ' DATE1', ' CLOSE_PRICE'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'CLOSE_PRICE'], 'pivot_value': 'CLOSE_PRICE', 'outfile': 'NSE_PRICE_DATA_CLOSE.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_CLOSE.csv
{'required':

/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nse_data = pd.concat([nse_data, pivot_data], ignore_index=True)
/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nse_data = pd.concat([nse_data, pivot_data], ignore_index=True)
/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:50: FutureWarning: The behavior of 

{'required': ['SYMBOL', ' SERIES', ' DATE1', ' DELIV_QTY'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'DELIV_QTY'], 'pivot_value': 'DELIV_QTY', 'outfile': 'NSE_PRICE_DATA_DELIV_QTY.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_DELIV_QTY.csv
{'required': ['SYMBOL', ' SERIES', ' DATE1', ' DELIV_PER'], 'renamed': ['SYMBOL', 'SERIES', 'Date', 'DELIV_PER'], 'pivot_value': 'DELIV_PER', 'outfile': 'NSE_PRICE_DATA_DELIV_PER.csv', 'logger': <Logger price_loger (DEBUG)>}
NSE_PRICE_DATA_DELIV_PER.csv


/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nse_data = nse_data.groupby('Date', as_index=False).first()
/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nse_data = pd.concat([nse_data, pivot_data], ignore_index=True)
/var/folders/pr/5qrnbjyd35g7js8x5y3q4g300000gn/T/ipykernel_49065/1831065742.py:54: PerformanceWarning: DataFrame is highly fragmented. 

True